In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from scipy.stats import ttest_ind, ttest_ind_from_stats
import matplotlib.pyplot as plt
import json

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory
path=[]
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        path.append(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# **Input Data into Table**

In [ ]:
data1=pd.read_csv(path[0])
data2=pd.read_csv(path[1])
data = pd.merge(left=data1,right=data2, left_on='id', right_on='movie_id')
data.head()

**Describe of Data**
* We have Budget, Popularity, Revenue, Runtime, Vote_average, Vote_count, Movie_id in Int64/ float64 datatype.
* movie_id and id can be omitted.
* homepage also need to be drop since it has too much missing value.
* original_title , title_x and title_y are the same so we only keep original_title column.
* Runtime has some missing value, I do mean imputation.

In [ ]:
data.describe()

In [ ]:
data.info()

In [ ]:
data.columns

In [ ]:
data=data.drop(['id','movie_id','homepage','title_x','title_y'],axis=1)
data['runtime'].fillna(np.mean(data['runtime']),inplace=True)
data

**Split the second table to find relationship between genre and revenue, vote_average**

In [ ]:
data2=data[['revenue','vote_average']]

In [ ]:
genre_file=data['genres'].apply(json.loads)
gen_table=[]
for each_movie in genre_file:
    gen_table.append([genre_name['name'] for genre_name in each_movie])

In [ ]:
data2['genre']=gen_table
data2

# 1. What areas have the most influence on revenue?
* Base on correlation matrix, we can see that budget, popularity and vote_count strongly affect the revenue of the movie.
* However, popularity and and vote_count also highliy correlated since the more vote you have that mean you have a lot a audience.

In [ ]:
data_1=data[["revenue","budget","popularity","runtime","vote_average","vote_count"]]
corr = data1.corr()
corr.style.background_gradient(cmap='coolwarm')

# 2. How is a movie’s revenue and average score affected by its genre?

In [ ]:
data2

# 3. What influence does release date have on revenue?
* There is little evidence to prove that release date( quarterly) affect the revenue of a movie
* However, we can see that quarter have more movies released than others.

In [ ]:
import datetime as dt
data3=data[['release_date','revenue']]
data3['release_date']= pd.to_datetime(data3['release_date'])
data3['quarter'] = data3['release_date'].dt.quarter
data3

In [ ]:
plt.hist(data3['quarter'])

In [ ]:
def check_weekday(date):
      
    # computing the parameter date
    # with len function
    res=len(pd.bdate_range(date,date))
      
    if res == 0 :
        return 0
    else:
        return 1

In [ ]:
data3['weekday']=data3['release_date'].dt.dayofweek
data3['isWeekend']=(data3['weekday']>=5).astype(int) #0 for weekday and 1 for weekend

In [ ]:
data3

In [ ]:
plt.hist(data3['isWeekend'])

In [ ]:
#data3['weekday'].apply(pd.value_counts).plot.pie(y='weekday', figsize=(5, 5))

In [ ]:
corr = data3.corr()
corr.style.background_gradient(cmap='coolwarm')